##### Building a Chatbot

This chatbot will only use the language model to have a conversation, There are several other related concepts that we will work on in future:
1. Conversational RAG: Enable a chatbot experience over an external source of data
2. Agents: Build a Chatbot that can take actions

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.3-70b-versatile",groq_api_key=groq_api_key)

c:\Users\HP\anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000024C59120340>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024C590576D0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Gunjan and I am looking for an SDE-AI role")])
#This will give me an AIMessage which is basically a response from my LLM model


AIMessage(content="Nice to meet you, Gunjan! I'd be happy to help you with your job search for an SDE-AI (Software Development Engineer - Artificial Intelligence) role.\n\nTo get started, could you please tell me a bit more about your background and what you're looking for in an SDE-AI role? For example:\n\n* What's your current level of experience (e.g., fresher, 2-5 years, 5+ years)?\n* What type of AI/ML technologies are you interested in working with (e.g., computer vision, NLP, deep learning)?\n* Are you looking for a specific industry or domain (e.g., healthcare, finance, autonomous vehicles)?\n* Do you have any preferred locations for the role (e.g., specific cities, remote work)?\n* What are your salary expectations?\n\nFeel free to share any other relevant details about your qualifications, skills, and interests. I'll do my best to provide you with helpful guidance and resources for your job search!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens'

In [5]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, My name is Gunjan and I am looking for an SDE-AI role"),
    AIMessage(content="Hello Gunjan! Nice to meet you. It's great that you're looking for an SDE-AI (Software Development Engineer - Artificial Intelligence) role. That's a fascinating field, and I'm sure you must have a strong background in computer science, math, and AI/ML concepts.\n\nTo better understand your requirements, could you please tell me a bit more about your background, skills, and what you're looking for in an SDE-AI role? For example:\n\n* What type of AI/ML technologies are you interested in working with (e.g., computer vision, NLP, deep learning)?\n* Do you have any specific industry or domain in mind (e.g., healthcare, finance, autonomous vehicles)?\n* What are your preferred programming languages and tools?\n* Are you looking for a role in a specific location or are you open to remote work?\n* Do you have any specific experience or qualifications that you think would be relevant to an SDE-AI role?\n\nFeel free to share as much or as little information as you'd like, and I'll do my best to help you explore SDE-AI opportunities that fit your interests and skills!"),
    HumanMessage(content="Hi, What;s my name and what am I looking for?")
]) #Here i am hardcoding the AIMessage, the message i got from the previous cell
#I am then asking the AI whether it can remember the message i gave it before

AIMessage(content='Your name is Gunjan, and you are looking for an SDE-AI (Software Development Engineer - Artificial Intelligence) role.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 311, 'total_tokens': 338, 'completion_time': 0.032351481, 'completion_tokens_details': None, 'prompt_time': 0.015378982, 'prompt_tokens_details': None, 'queue_time': 0.056355737, 'total_time': 0.047730463}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c18aa-ed99-7c81-8f75-ebed71dcc582-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 311, 'output_tokens': 27, 'total_tokens': 338})

### Message History
We can use Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

#Now as different different users and simultaneously interacting with the chatbot we need to make sure that one session is completely different from the other session
#We will create a function for that with the parameter as session id which is a string
#The resturn type of this function would be BaseChatMessageHistory--->whatever chat history is created it is imported from this library

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory: #this session id will be used to distinguish between sessions
    if session_id not in store:
        store[session_id]=ChatMessageHistory() #this is an object of chat message history
        #we are storing the session id in the store dictionary
        #With respect to the session id value i am initializing a chat message history
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)
#this ensures that we can interact with our LLM model based on our chat history

In [7]:
config={"configurable":{"session_id":"chat1"}} #here i am hardcoding the session id

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Gunjan and I am looking for an SDE-AI role")],
    config=config
)

In [9]:
response.content

"Hi Gunjan, nice to meet you! Congratulations on exploring opportunities in the field of Software Development Engineering - Artificial Intelligence (SDE-AI). That's a fascinating and in-demand role.\n\nTo better understand your requirements and preferences, could you please share some details about your background and what you're looking for in an SDE-AI role? For example:\n\n* What's your current level of experience (entry-level, mid-level, senior-level)?\n* What specific areas of AI interest you the most (e.g., machine learning, natural language processing, computer vision)?\n* Are you looking for a role in a particular industry (e.g., healthcare, finance, tech)?\n* Do you have any preferred locations or are you open to remote work?\n* What are your salary expectations?\n\nFeel free to share as much or as little information as you'd like, and I'll do my best to help you explore SDE-AI opportunities that fit your goals and aspirations!"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

AIMessage(content='Your name is Gunjan.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 262, 'total_tokens': 269, 'completion_time': 0.006961073, 'completion_tokens_details': None, 'prompt_time': 0.013635419, 'prompt_tokens_details': None, 'queue_time': 0.057270381, 'total_time': 0.020596492}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c18aa-f164-7c31-89cc-36b2a1066456-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 262, 'output_tokens': 7, 'total_tokens': 269})

In [11]:
#Change the config--->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)
response.content

"I don't know your name. I'm a large language model, I don't have any information about you, including your name. I'm a text-based AI assistant, and our conversation just started, so I don't have any prior knowledge about you. If you'd like to share your name, I'd be happy to chat with you and use it in our conversation!"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi, my name if Gina")],
    config=config1
)
response.content

"Hi Gina! It's nice to meet you! I'll make sure to address you by your name in our conversation. How's your day going so far? Is there something I can help you with or would you like to chat about anything in particular?"

In [13]:

response=with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)
response.content

'I remember! Your name is Gina.'

##### Prompt templates

Prompt templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make it a bit more complicated. First let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer all questions to the best of your ability"),
    MessagesPlaceholder(variable_name="messages")
])
chain=prompt|model

In [15]:
#Whatever human message i give needs to be given in a key value pair and the key name should be "messages"
#Earlier i was not using any message placeholder so i was giving a list of messages like human message, system message, ai message
#Here I have specified that I will use a message placeholder called as messages
chain.invoke({"messages":[HumanMessage(content="Hi, My name is Gunjan")]})

AIMessage(content="Hello Gunjan! It's nice to meet you. Is there something I can help you with or would you like to chat? I'm here to assist you with any questions or topics you'd like to discuss. How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 57, 'total_tokens': 109, 'completion_time': 0.107014351, 'completion_tokens_details': None, 'prompt_time': 0.00522166, 'prompt_tokens_details': None, 'queue_time': 0.053405547, 'total_time': 0.112236011}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c18aa-f4f0-72d3-9eac-a5954c9d9cb4-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 57, 'output_tokens': 52, 'total_tokens': 109})

In [16]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [17]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="hi my name in gunjan")],
    config=config
)

In [18]:
response

AIMessage(content="Hello Gunjan, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 56, 'total_tokens': 83, 'completion_time': 0.054222771, 'completion_tokens_details': None, 'prompt_time': 0.002701883, 'prompt_tokens_details': None, 'queue_time': 0.053248976, 'total_time': 0.056924654}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c18aa-f5eb-7332-b65e-00a2da97b08f-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 56, 'output_tokens': 27, 'total_tokens': 83})

In [19]:
prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain=prompt|model

In [20]:
response=chain.invoke({"messages":[HumanMessage(content="Hi, my name is Gunjan")],"language":"Hindu0"})
response.content

'नमस्ते गुंजन, मैं आपकी सहायता के लिए तैयार हूँ। क्या मैं आपकी किसी प्रश्न या समस्या का समाधान कर सकता हूँ?'

In [31]:
#Let's wrap this more complicated chain in message history class
#We need to specify correct key to use and save the chat history
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [34]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"language":"Hindi","messages":[HumanMessage(content="Whats my name")]},
    config=config
)
response.content

'आपका नाम गुंजन है।'

#### Managing Conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of messages you are passing in.
'trim_messages' helps to reduce how many messages we are sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [43]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=50,
    strategy="last", #start counting tokens from the last message
    token_counter=model,
    include_system=True, #include system message
    allow_partial=False, #i dont want partial information
    start_on="human" #start from human conversation

)
messages=[
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm Gunjan"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="Having fun?"),
    AIMessage(content="yes!"),
]

In [44]:
trimmer.invoke(messages) #it will take 50 tokens from last message

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='whats 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='Having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

In [46]:
#how to pass trimmer in a chain
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer) #using this we can retrieve entire messages available in prompt template
    |prompt
    |model
)
response=chain.invoke({
    "messages":messages + [HumanMessage(content="what ice cream do i like")],
    "language":"English"}
)
response.content

'You like vanilla ice cream!'

In [47]:
response=chain.invoke({
    "messages":messages + [HumanMessage(content="what math problem did i ask?")],
    "language":"English"}
)
response.content

'You asked me to solve the math problem "2+2".'

In [48]:
#Lets wrap this in the message history
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config={"configurable":{"session_id":"chat5"}}

In [50]:
response=with_message_history.invoke({
    "messages":messages + [HumanMessage(content="what math problem did i ask?")],
    "language":"English"},
    config=config
)
response.content

'You asked me to solve the math problem "2+2". The answer was 4.'